# GraphAI client

## Introduction

Welcome to `graphai-client`. This package is designed to streamline programmatic interaction with the [GraphAI API](https://graphai.epfl.ch/), taking care of logins, authentication tokens, retries, and asynchronous as well as synchronous endpoints.

This notebook will take you through a typical use case of the API through the client. Remember that in order to access any of the endpoints, you need to have a GraphAI account, which will give you access to some or all of the endpoint groups.

## Installation

### Installing the package

Default installation:

`pip install git+https://github.com/epflgraph/graphai-client.git`

Editable installation (pip version >= 21.3):

`pip install -e git+https://github.com/epflgraph/graphai-client.git`

### Preparing to run this notebook

In order to run this notebook, you need to have a GraphAI account. Create a `config` directory in the same directory as this notebook, create a `graphai-api.json` file within it, and set up its contents as follows:

```json
{
  "host": "https://graphai.epfl.ch",
  "port": 443,
  "user": "PUT_YOUR_USERNAME_HERE",
  "password": "PUT_YOUR_PASSWORD_HERE"
}
```

In [1]:
credentials_path = 'config/graphai-api.json'

In [2]:
from graphai_client.client import login

## Use

### Example 1: Translating text

**This section requires your account to have access to the `translation` endpoints. If you encounter a permission error, contact the administrator.**

Let's say you have some text in French that you want to translate to English. Before you get started, you need to log in using the `login` function:

In [3]:
login_info = login(graph_api_json=credentials_path)

Now that you're logged in, let's discuss the translation function. The `graphai_client.client_api.translation.translate_text` function handles translation for you. All you need to provide are the text, the source and target languages, and the login info you just obtained.

This function is part of the direct API functionalities, all of which are found in the `graphai_client.client_api` subpackage.

At the time of this notebook's creation, the supported languages are EN-FR, FR-EN, IT-EN, and DE-EN.

In [4]:
from graphai_client.client_api.translation import translate_text
french_text = "Elle vend des coquillages au bord de la mer."
translated_text = translate_text(text=french_text, source_language='fr', target_language='en', login_info=login_info)

[2024-10-07 16:42:03] [GRAPHAI] [TRANSLATE] [PROCESSING] extracting en translation from fr text (44 characters)...
[2024-10-07 16:42:03] [GRAPHAI] [TRANSLATE] [SUCCESS] en translation has been extracted from fr text (44 characters) (already done in the past)


In [5]:
translated_text

' She sells shellfish by the sea.'

Not exactly "she sells seashells by the seashore", but close enough!

Let's see another translation, this time from English.

In [6]:
english_text = "I could make so many requests to the API and I still would not reach the rate limit."
translated_text = translate_text(text=english_text, source_language='en', target_language='fr', login_info=login_info)

[2024-10-07 16:42:03] [GRAPHAI] [TRANSLATE] [PROCESSING] extracting fr translation from en text (84 characters)...
[2024-10-07 16:42:03] [GRAPHAI] [TRANSLATE] [SUCCESS] fr translation has been extracted from en text (84 characters) (already done in the past)


In [7]:
translated_text

" Je pourrais faire autant de demandes à l'API et je n'atteindrais toujours pas la limite de taux."

### Example 2: Processing a video

**This section requires your account to have access to the `video`, `voice`, and `image` endpoints. If you encounter a permission error, contact the administrator.**

Since video processing is the central functionality of GraphAI, `graphai-client` provides you with a singular function to directly and fully process a video: from downloading to slide and audio extraction and finally to OCR and audio transcription. This function is `graphai_client.client.process_video`.

This function receives several flags and inputs. Some of the most important ones are:
* `analyze_audio`: if set, the audio is extracted and transcribed. `True` by default.
* `analyze_slides`: if set, slides are extracted and OCR is performed on them. You need to provide a Google API token for this endpoint to work. `True` by default.
* `destination_languages`: list of languages to translate the results to. `['en', 'fr']` by default.

In [8]:
from graphai_client.client import process_video
url= 'http://api.cast.switch.ch/p/113/sp/11300/serveFlavor/entryId/0_00gdquzv/v/2/ev/3/flavorId/0_i0v49s5y/forceproxy/true/name/a.mp4'
# In order to enable slide analysis, provide your own Google Vision API token below.
# We cannot provide our own since this notebook is publicly available.
google_api_token = None
video_info = process_video(url, analyze_slides=google_api_token is not None, login_info=login_info, google_api_token=google_api_token)

[2024-10-07 16:42:03] [GRAPHAI] [DOWNLOAD VIDEO] [PROCESSING] extracting file from http://api.cast.switch.ch/p/113/sp/11300/serveFlavor/entryId/0_00gdquzv/v/2/ev/3/flavorId/0_i0v49s5y/forceproxy/true/name/a.mp4...
[2024-10-07 16:42:11] [GRAPHAI] [DOWNLOAD VIDEO] [SUCCESS] file (0.8 MB) has been extracted from http://api.cast.switch.ch/p/113/sp/11300/serveFlavor/entryId/0_00gdquzv/v/2/ev/3/flavorId/0_i0v49s5y/forceproxy/true/name/a.mp4 (already done in the past) (all are active) (all are fingerprinted)
[2024-10-07 16:42:11] [GRAPHAI] [EXTRACT AUDIO] [PROCESSING] extracting audio from 169770835520421902463099.mp4...
[2024-10-07 16:42:12] [GRAPHAI] [EXTRACT AUDIO] [SUCCESS] audio has been extracted from 169770835520421902463099.mp4 (already done in the past) (all are active) (all are fingerprinted)
[2024-10-07 16:42:12] [GRAPHAI] [AUDIO FINGERPRINT] [PROCESSING] extracting fingerprint from 169770835520421902463099.mp4_audio.ogg...
[2024-10-07 16:42:12] [GRAPHAI] [AUDIO FINGERPRINT] [SUCCE

Here we can see that the video was downloaded, its audio was extracted, and the subtitles were generated. The results also include a variety of information on the video and audio streams of the video file, plus the internal video and audio tokens. If you wish to further process the same file, you will have to use these tokens to refer to the file you have made prior requests for.

In [9]:
video_info

{'url': 'http://api.cast.switch.ch/p/113/sp/11300/serveFlavor/entryId/0_00gdquzv/v/2/ev/3/flavorId/0_i0v49s5y/forceproxy/true/name/a.mp4',
 'video_size': 881966,
 'video_token': '169770835520421902463099.mp4',
 'slides': None,
 'slides_language': None,
 'subtitles': [{'id': 0,
   'start': 0.0,
   'end': 5.0,
   'en': 'These subtitles have been generated automatically\nBacteria GFP Expression',
   'fr': 'Ces sous-titres ont été générés automatiquement\nExpression des bactéries GFP'},
  {'id': 1,
   'start': 5.0,
   'end': 17.0,
   'en': 'The data consists of bacteria images acquired in face contrast and fluorescence across three different conditions, A, B, and C, with five replicates per condition.',
   'fr': 'Les données consistent en des images de bactéries acquises en contraste facial et en fluorescence dans trois conditions différentes, A, B et C, avec cinq répétitions par condition.'},
  {'id': 2,
   'start': 17.0,
   'end': 24.0,
   'en': "The fluorescent channel represents a GFP-

### Example 3: Getting word embeddings for text

**This section requires your account to have access to the `translation` endpoints. If you encounter a permission error, contact the administrator.**

Another group of endpoints provided through the client is `embedding`, which allows you to embed a given text as a vector. Here's an example of how to use this functionality:

In [10]:
from graphai_client.client_api.embedding import embed_text
text_to_embed = "The Graph project at the Federal Institute of Technology in Lausanne"
result = embed_text(text_to_embed, login_info)

[2024-10-07 16:42:12] [GRAPHAI] [EMBED] [PROCESSING] extracting embedding from text (68 characters)...
[2024-10-07 16:42:13] [GRAPHAI] [EMBED] [SUCCESS] embedding has been extracted from text (68 characters) (already done in the past)


In [11]:
import numpy as np
result_vector = np.array(result)
result_vector.shape

(384,)